In [1]:
from typing import List
from collections import deque, defaultdict


class UnionFind:
  def __init__(self, n):
    self.parent = list(range(n))
    self.rank = [0] * n
    self.component_cost = [-1] * n

  def find(self, u):
    if self.parent[u] != u:
      self.parent[u] = self.find(self.parent[u])
    return self.parent[u]

  def union(self, u, v, weight):
    root_u = self.find(u)
    root_v = self.find(v)

    if root_u != root_v:
      if self.rank[root_u] > self.rank[root_v]:
        self.parent[root_v] = root_u
        self.component_cost[root_u] &= self.component_cost[root_v] & weight
      elif self.rank[root_u] < self.rank[root_v]:
        self.parent[root_u] = root_v
        self.component_cost[root_v] &= self.component_cost[root_u] & weight
      else:
        self.parent[root_v] = root_u
        self.rank[root_u] += 1
        self.component_cost[root_u] &= self.component_cost[root_v] & weight

  def set_component_cost(self, u, cost):
    root = self.find(u)
    self.component_cost[root] &= cost

  def get_component_cost(self, u):
    root = self.find(u)
    return self.component_cost[root]


class Solution:
  def minimumCost(self, n: int, edges: List[List[int]], query: List[List[int]]) -> List[int]:

    uf = UnionFind(n)
    graph = defaultdict(list)
    for u, v, w in edges:
      graph[u].append((v, w))
      graph[v].append((u, w))

    visited = [False] * n
    for node in range(n):
      if not visited[node]:
        self.compute_component_cost(node, graph, uf, visited)

    result = []
    for s, t in query:
      if uf.find(s) == uf.find(t):
        result.append(uf.get_component_cost(s))
      else:
        result.append(-1)

    return result

  def compute_component_cost(self, start, graph, uf, visited):
    queue = deque([(start, -1)])
    visited[start] = True
    uf.set_component_cost(start, -1)

    while queue:
      node, cost = queue.popleft()
      root = uf.find(start)

      for neighbor, weight in graph[node]:
        new_cost = cost & weight if cost != -1 else weight
        uf.set_component_cost(root, new_cost)

        if not visited[neighbor]:
          visited[neighbor] = True
          uf.union(start, neighbor, new_cost)
          queue.append((neighbor, new_cost))

In [2]:
n = 5
edges = [[0, 1, 7], [1, 3, 7], [1, 2, 1]]
query = [[0, 3], [3, 4]]
Solution().minimumCost(n=n, edges=edges, query=query)

[1, -1]

In [3]:
n = 3
edges = [[0, 2, 7], [0, 1, 15], [1, 2, 6], [1, 2, 1]]
query = [[1, 2]]
Solution().minimumCost(n=n, edges=edges, query=query)

[0]